### Set up

In [1]:
?- cd('~/work'), ['dec'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

## Termination of expectation rules and associated expectations

Expectation rules should be subject to change if necessary. Specific events can be used to initiate and terminate expectation rules. As expectation rules lead to expectations holding when their conditions have been fulfilled, we also want to specify how termination of an expectation rule will affect the associated expectations.

For example, we could have an expectation rule which states that all patients who had admitted to hospital will eventually be discharged. The expectation that a patient would eventually be discharged would be triggered when they were admitted, and fulfilled upon their dischargement. This expectation would not be fulfilled for any patients who passed away in hospital. At a point where multiple patients are currently in hospital and have not yet been discharged, our expectation rule may be terminated by an event. It must then be decided whether the expectation of eventual discharge should continue to be applied to existing patients, or removed from all current patients, in addition to not applying to future admissions.

### Example Expectation Rule (dependent)

We use an expectation rule with the form **exp_rule(Condition, Outcome, Status, Message))**. **Condition** refers to the triggering condition for the resulting expectation and **Outcome** is the state that needs to be reached for the expectation to be fulfilled. If **Status** is set to *independent*, then expectations resulting from our expectation will remain even if the expectation rule is deleted. **Message** is a more user friendly message describing the expectation rule. Currently, **Message** is used as a unique identifier to facilitate easier termination of expectation rules.

In our example expectation rule, we declare that the starting condition for our expectation is the admission of a patient, that the resulting expected outcome is that the patient will eventually be discharged, that the resulting expectations shall be dependent, and use the message "Patients must eventually be discharged" to describe the expectation rule.

In [2]:
% File: expectationRule.pl
initially(exp_rule(happ(admission(Patient)), eventually(happ(discharge(Patient))), dependent, "Patients must eventually be discharged")).

### Example Rule Termination

An **optionalDischarge** event can be used to terminate our expectation rule.

In [3]:
% File: expRuleTermination.pl
terminates(optionalDischarge, exp_rule(_,_,_, "Patients must eventually be discharged"), _).

### Example Narrative

Patient 123 is admitted at time period 1, while patients 124 and 125 are both admitted at time period 2. The **optionalDischarge** event occurs at time period 3. 

In [4]:
% File: narrative.pl
happensAtNarrative(admission(123),1).
happensAtNarrative(admission(124),2).
happensAtNarrative(admission(125),2).
happensAtNarrative(optionalDischarge, 3).

At time period zero, we can see that our expectation rule holds.

In [5]:
?- tick(0).
?- holdsAt(H,0).

true.
H = exp_rule(happ(admission(_1674)), eventually(happ(discharge(_1674))), dependent, b'Patients must eventually be discharged') .

At time period one, our expectation rule continues to hold. The expectation that Patient 123 will eventually be admitted also holds. The 6 argument expectation has been hidden for readibility.

In [6]:
?- tick(1).
?- happensAt(Event, 1).
?- holdsAt(H,1), H \= exp(_,_,_,_,_,_).
?- holdsAt(exp_rule(_,_,_,Message),1).

true.
Event = admission(123) .
H = exp_rule(happ(admission(_1686)), eventually(happ(discharge(_1686))), dependent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(123))), dependent, b'Patients must eventually be discharged') .
Message = b'Patients must eventually be discharged' .

At time period two, we expect that all three patients will eventually be discharged. 

In [7]:
?- tick(2).
?- happensAt(Event, 2).
?- holdsAt(H,2), H \= exp(_,_,_,_,_,_).


true.
Event = admission(124) ;
Event = admission(125) .
H = exp_rule(happ(admission(_1686)), eventually(happ(discharge(_1686))), dependent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(123))), dependent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(124))), dependent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(125))), dependent, b'Patients must eventually be discharged') .

At time period three, the **optionalDischarge** event occurs. At this time period, the expectation rule and resulting expectations still hold.

In [8]:
?- tick(3).
?- happensAt(Event, 3).
?- holdsAt(H,3), H \= exp(_,_,_,_,_,_).


true.
Event = optionalDischarge .
H = exp_rule(happ(admission(_1686)), eventually(happ(discharge(_1686))), dependent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(123))), dependent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(124))), dependent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(125))), dependent, b'Patients must eventually be discharged') .

At time period four, the expectation rule no longer holds, as it was terminated by the **optionalDischarge** event. As the status of the expectation rule was set to **dependent**, the expectations that patients 123, 124 and 125 will eventually be discharged also no longer exist.

In [9]:
?- tick(4).
?- happensAt(Event, 4).
?- holdsAt(H,4).


true.
false.
false.

### Independent Expectation Scenario

Let us instead consider a scenario in which the expectation are not dependent on their expectation rule. Note that the third argument of the new expectation rule is **independent** rather than **dependent**. We use the **newExpRule** event at time period five to initiate our new expectation rule. Patient 126 is admitted at time period six, and patients 127 and 128 are admitted at time period seven. Also at time period seven, the **optionalDischarge** event occurs which terminates our expectation rule.  

In [10]:
% File: dependentScenario.pl
initiates(newExpRule(Condition,Outcome,Status,Message), exp_rule(Condition,Outcome,Status,Message),_).
happensAtNarrative(newExpRule(happ(admission(Patient)), eventually(happ(discharge(Patient))), independent, 
"Patients must eventually be discharged"),5).
happensAtNarrative(admission(126),6).
happensAtNarrative(admission(127),7).
happensAtNarrative(admission(128),7).
happensAtNarrative(optionalDischarge, 7).

Our expectation rule holds from time period five onwards.

In [11]:
?- tick(5).
?- happensAt(Event, 5).
?- holdsAt(H,5).


true.
Event = newExpRule(happ(admission(_1620)), eventually(happ(discharge(_1620))), independent, b'Patients must eventually be discharged') .
false.

The expectation that patient 126 will eventually be discharged has been correctly triggered by the expectation rule.

In [12]:
?- tick(6).
?- happensAt(Event, 6).
?- holdsAt(H,6), H \= exp(_,_,_,_,_,_).


true.
Event = admission(126) .
H = exp_rule(happ(admission(_1686)), eventually(happ(discharge(_1686))), independent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(126))), independent, b'Patients must eventually be discharged') .

Similarly, expectations also hold for patients 127 and 128 at time period seven. Also at this time period, the **optionalDischarge** event takes place.

In [13]:
?- tick(7).
?- happensAt(Event, 7).
?- holdsAt(H,7), H \= exp(_,_,_,_,_,_).


true.
Event = admission(127) ;
Event = admission(128) ;
Event = optionalDischarge .
H = exp_rule(happ(admission(_1686)), eventually(happ(discharge(_1686))), independent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(126))), independent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(127))), independent, b'Patients must eventually be discharged') ;
H = exp(eventually(happ(discharge(128))), independent, b'Patients must eventually be discharged') .

At time period eight, our expectation rule no longer holds. However, because our expectations are independent of the rule, they continue to hold.

In [14]:
?- tick(8).
?- holdsAt(exp_rule(Condition,Outcome, Status, Message),8).
?- holdsAt(H,8), H \= exp(_,_,_,_,_,_).

true.
false.
H = exp(eventually(happ(discharge(126))), independent, _1676) ;
H = exp(eventually(happ(discharge(127))), independent, _1676) ;
H = exp(eventually(happ(discharge(128))), independent, _1676) .

So, we can set the status of expectation rules to independent or dependent in order to control whether expectations are terminated along with their parent expectation rule. A text message can be used to describe expectation rules in a more user friendly manner, and used to identify expectation rules.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>